In [1]:
import seaborn as sns
import pandas as pd
from sklearn.metrics import recall_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Lasso,RidgeCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import median_absolute_error

## Class ile

In [58]:
 class optimizer:
    # First I add some inputs via init method
    def __init__(self,model,param_space,fold,stop=4,tuning='grid_search'):
        sklearn= __import__('sklearn')
        self.median_err= sklearn.metrics.median_absolute_error
        self.np= __import__('numpy')
        self.pd= __import__('pandas')
        self.comb= __import__("itertools").combinations
        self.split= sklearn.model_selection.train_test_split
        self.minmax= sklearn.preprocessing.MinMaxScaler
        self.polyfeat= sklearn.preprocessing.PolynomialFeatures
        self.grid= sklearn.model_selection.GridSearchCV
        # model that we'll use
        self.model=model
        # num of iteration that will happen in each iteration for each param_space
        self.stop=stop
        # we use 2 type of tuning
        self.tuning= tuning
        self.param_space=param_space
        self.fold=fold
        self.ch=0
    # we split target during cleaning
    def target_ch(self,x):
        t=x.split()
        if t[2].lower()=='night':
            return float(t[0]),t[2].lower()
        elif t[2].lower()=='month':
            return float(t[0]),t[2].lower()
        elif t[2].lower()=='week':
            return float(t[0]),t[2].lower()
    def prep(self,x,t=0):
        # t is a flag that used to determine whether it is train or test
        # Here we get columns that will used for polynomial feature and we use it for test data
        if t==0:
            self.col_poly= []
            for i in x.columns:
                if x[i].nunique()>10 and x[i].dtype=='float64':
                    self.col_poly.append(i)
        poly= self.polyfeat(interaction_only=True)
        train_poly= pd.DataFrame(poly.fit_transform(x[self.col_poly])[:,1:],columns=poly.get_feature_names_out(self.col_poly)[1:],\
                                 index=x.index)
        col_pol=poly.get_feature_names_out(self.col_poly)[1:]
        x= x.drop(columns=self.col_poly,axis=1)
        x= pd.concat([x,train_poly],axis=1)
        # Minmaxscaler
        sc= self.minmax()
        x[col_pol]= sc.fit_transform(x[col_pol])
        # Categoric data encoding
        col_dummies= x.select_dtypes('O').columns
        x= pd.get_dummies(x,columns=col_dummies)
        if t==0:
            # Some columns located in train not in test and vice versa
            self.tr_cols=x.columns
        if t==1:
            # Create empty dataframe with train cols and add test features that common otherwise '0' column
            test_cols= x.columns
            t_x= self.pd.DataFrame(columns=self.tr_cols)
            for col in self.tr_cols:
                    if col in  test_cols:
                        t_x[col]= x[col]
                    else:
                        t_x[col]=0
            x= t_x
        return x
    # return score of model on test data
    def score(self):
        print('median error: {0}'.format(self.median_err(self.y_test,self.best_model_.predict(self.x_test))))
    def fit(self,df):
        # apply target_ch function
        res= df['prices'].apply(self.target_ch)
        df['prices']= res.apply(lambda x: float(x[0]))
        # Drop columns that contains 95% same value
        for i in df.columns:
                if df[i].value_counts().iloc[0]>=df.shape[0]*.95:
                    df= df.drop(columns=i,axis=1)
        self.x= df[[i for i in df.columns if i!='prices']]
        self.y= df['prices']
        self.x_train,self.x_test,self.y_train,self.y_test= self.split(self.x,self.y)
        # BAX
        self.x_train= optimizer.prep(self,self.x_train,t=0)
        self.x_test= optimizer.prep(self,self.x_test,t=1)

        if self.tuning=='binary_search':
            self.best_params_= optimizer.hyp_optimizer(self)
            # Pass parameters as kwargs to model
            self.best_model_= self.model(**self.best_params_).fit(self.x_train,self.y_train)
        elif self.tuning=='grid_search':
            # best params and best model is created in gridsearch
            optimizer.grid_search(self)
        # get score of model on test data
        optimizer.score(self)
        print(self)
        
    # If we have uncleaned data we want to clean data before predict for that we apply .predict()
    # directly without defining .best_model_
    def predict(self,x,clean='yes'):
        mdl= self.best_model_
        if clean!='yes':
            x=optimizer.prep(self,x,t=1)
        return mdl.predict(x)
    
    # This function train data for given parameters with num of folds
    def yoxla(self,arr):
        # we get number of parameters in hyp_optimizer and check its score in yoxla
        if self.ch==1:
            model= self.model(**{list(self.param_space.keys())[i]:abs(arr[i]) if type(arr[i])!=str else arr[i] for i in range(len(arr))})
        else:
            model= self.model(**{list(self.param_space.keys())[i]:int(abs(arr[i])) if type(arr[i])!=str else arr[i] for i in range(len(arr))})
            
        score=0
        # loop over folds
        for _ in range(self.fold):
            # We select (fold-1)/fold ratio of data randomly
            ind= self.np.random.choice(self.x_train.index,int(((self.fold-1)/self.fold)*len(self.x_train)),replace=False)
            # we get not_ind for calculating score on dataset
            not_ind=[i for i in self.x_train.index if i not in ind]
            kf_x= self.x_train.loc[ind,:]
            kf_y= self.y_train.loc[ind]
            model.fit(kf_x,kf_y)
            # calculate score for each fold and calculate average 
            score+=self.median_err(self.y_train.loc[not_ind],model.predict(self.x_train.loc[not_ind,:]))
        return score/self.fold
    def grid_search(self):
        gridsearch =self.grid(self.model(),self.param_space,cv=self.fold,error_score='raise')
        print(gridsearch)
        gridsearch.fit(self.x_train,self.y_train)
        self.best_params_=  gridsearch.best_params_
        self.best_model_= gridsearch.best_estimator_
        return self
    def medians(self,R):
        if len(R[0])>2:
            if 'alphas' in list(self.param_space.keys()):
                self.ch=1
                return [self.np.median(i)  for i in R]
            else:
                return [round(self.np.median(i))  for i in R]
                
        else:
            return [round(i[0]+float((i[1]-i[0]+1)/2)) if (i[1]-i[0])%2==0 \
                else round((2*i[0]+float((i[1]-i[0])/2)+(float((i[1]-i[0])/2))+1)/2)   for i in R]
            
    def hyp_optimizer(self):
        # We could have string parameters we calculate them after this process
        T= [i  for i in self.param_space.values() if type(i[0])!=str]
        Tm= T.copy()
        # We take median for each given range of parameter
        Lc = optimizer.medians(self,T)
        ind=0
        lim=0
        cat_g=0
        # there is limitation for num of loop limt count this
        limt= [0 for i in range(len(T))]
        optim= Lc
        score=-1
        while True:
            t=0
            # In Binary logic it check mid point, left and right end points, but here we have range and that end points can give
            # us false direction that's why I calculate it for  mid point mid+ sqrt(mid) and mid- sqrt(mid)
            # Because our aim is to just find next direction 
            L= optimizer.medians(self,Tm)
            if max(optimizer.yoxla(self,[float(L[i]+L[i]**.5) if i==ind else float(L[i]) for i in range(len(L))]),\
                   optimizer.yoxla(self,[float(L[i]-L[i]**.5) if i==ind else float(L[i]) for i in range(len(L))] ),\
                         optimizer.yoxla(self,L ))==optimizer.yoxla(self,[float(L[i]+L[i]**.5) if i==ind else float(L[i]) for i in range(len(L))] ):
                
                                          # if current score is smaller than yoxla() replace it in Tm and optim
                            if score<optimizer.yoxla(self,[float(L[i]+L[i]**.5) if i==ind else float(L[i]) for i in range(len(L))] ):
                            
                                              Tm[ind][0]= L[ind]
                                              # with t we check whether  optimal change or not 
                                              t=1
                                              # we update new parameters in L
                                              L= optimizer.medians(self,Tm)
                                              
                                              optim= [i for i in L.copy()]
                                              
                                              score= optimizer.yoxla(self,[float(L[i]+L[i]**.5) if i==ind else float(L[i]) for i in range(len(L))] )
            if max(optimizer.yoxla(self,[float(L[i]+L[i]**.5) if i==ind else L[i] for i in range(len(L))] ),\
                   optimizer.yoxla(self,[float(L[i]-L[i]**.5) if i==ind else L[i] for i in range(len(L))] ),\
                         optimizer.yoxla(self,L ))==optimizer.yoxla(self,[float(L[i]-L[i]**.5) if i==ind else float(L[i]) for i in range(len(L))] ):
                                          if score>optimizer.yoxla(self,[float(L[i]-L[i]**.5) if i==ind else float(L[i]) for i in range(len(L))] ):
                                              Tm[ind][1]= L[ind]
                                              t=1
                                              L= optimizer.medians(self,Tm)
                                              
                                              optim= [i for i  in L.copy()]

                                              score= optimizer.yoxla(self,[float(L[i]-L[i]**.5) if i==ind else float(L[i]) for i in range(len(L))] )
            if (ind==0 and t==0) or (ind==0 and lim==self.stop):
                # for first step we calculate yoxla() for each combination but in other keys we just change one time and
                # return to first step
                lim=-1
                # We already get best variant for checked params so we can back to previous type
                Tm[0]= T[0]
                br=0
                for k in range(ind,len(L)-1):
                    ind=k
                    if ind>=len(L):
                        br=1
                        break
                    else:
                        # increase limt for each parameter
                        if limt[ind+1]<self.stop and lim<self.stop:
                            ind+=1
                            br=2
                            break
                if ind==0 and br==0:
                    break
                if br==1:
                    # this means we are done, we dont have parameter to check any more 
                    break
                if br==2:
                    # it means we incread ind to next parameter because our current parameter already done
                    continue
            if ind!=0:
                # this is for 2rd and other parameter values
                limt[ind]+=1
                if t==1 and limt[ind]<self.stop:
                    Tm[0:ind]= T[0:ind]
                    ind=0
                    lim=-1

                else:
                        if lim>=self.stop:
                            break
                        br=0
                        for i in range(ind,len(T)-1):
                            ind=i
                            if ind>=len(L):
                                br=1
                                break
                            else:
                                if limt[ind+1]<=self.stop:
                                    ind+=1
                                    lim=-1
                                    break

                        if br==1 or sum(limt[1:])/len(limt)==self.stop:
                            break
            lim+=1
        if len(Tm)==0:
            Tm=L
       
        # Here we check for categorical parameter and find best on    
        y={}
        p=[]
        index=[]
        s=0
        for i,k in self.param_space.items():
            if type(k[0])==str:
                index.append(s)
                y[i]=k
                p+=k
            s+=1
        if len(index)>0:
            comb= list(self.comb(p,len(y.keys())))
            for i in range(len(comb)):
                sg=0
                for k in comb[i]:
                    for j in y.values():
                        if k in j:
                            sg+=1
                            if sg>=2:
                                del comb[i]


            mak=-1
            for c in comb:
                ocop=optim.copy()
                for k in range(len(c)):
                    ocop.insert(index[k],c[k])
                    if optimizer.yoxla(self,ocop)>mak:
                        mak=optimizer.yoxla(self,ocop)
                        op=ocop.copy()
            optim=op.copy()

        ans={}
        for i in range(len(optim)):
            if type(optim[i])!=str:
                ans[list(self.param_space.keys())[i]]= abs(optim[i])
            else:
                ans[list(self.param_space.keys())[i]]= optim[i]
        return ans

In [65]:
df=pd.read_csv('airbnb_full.csv').iloc[:,1:]

In [60]:
param_space= {"alphas":[0.0001,0.001,0.01,0.1,1,3,5,7,10,15]}
op= optimizer(RidgeCV,param_space,fold=4,tuning='binary_search')
op.fit(df)

median error: 84.2734961781614


In [62]:
param_space={"max_depth":[2,4,6],"min_samples_split":[4,6,8],'max_features':['sqrt', 'log2']}
op= optimizer(DecisionTreeRegressor,param_space,fold=4,tuning='binary_search')
op.fit(df)

median error: 93.58191126279863


In [64]:
param_space= {"n_neighbors":[5,10,20,30,50]}
op= optimizer(KNeighborsRegressor,param_space,fold=4,tuning='grid_search')
op.fit(df)

GridSearchCV(cv=4, error_score='raise', estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': [5, 10, 20, 30, 50]})
median error: 74.88
